In [127]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from scipy.stats import mode
from tqdm.notebook import tqdm
from p_tqdm import p_map
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
df_combined = pd.read_csv('../../search_results/LOPQ+Both+resnet18+256+20.csv', index_col=None)
excluded_classes = ['0235_D01_S001', '0200_D01_S001', '0049_D01_S001', '0842_D01_S001', '0337_D01_S001', '0263_D01_S001', '0735_D01_S001', '0541_D01_S001', '0311_D01_S001']
len(excluded_classes)
df_combined = df_combined[~df_combined.package_actual.isin(excluded_classes)]
df_combined.package_actual.nunique()

989

In [ ]:
df_bad_samples = pd.read_csv('to_investigate.csv', index_col=None)

In [ ]:
df_selected = df_combined[df_combined.sample_actual.isin(df_bad_samples['sample_actual'].to_list())]

In [129]:
dfs = [x for _, x in df_selected.groupby('sample_actual')]

In [ ]:
df.columns

In [ ]:
df = dfs[0]

sample_actual = df.iloc[0].sample_actual
package_actual = df.iloc[0].package_actual
alg = df.algorithm.unique()[0]
df[df.algorithm == 'MI1']

In [130]:
# for df in dfs:
df = dfs[1]

sample_actual = df.iloc[0].sample_actual
package_actual = df.iloc[0].package_actual

alg_guesses = []
for alg in df.algorithm.unique():
    df_piv = df[df.algorithm == alg].pivot_table(
        index='package_predicted', columns='descriptor_actual', values='distance', fill_value=np.nan,
        aggfunc=[np.mean, np.median, 'size', sum, min, 'std', np.var]#, ]
    )
    df_piv = df_piv.reset_index()

    df_rank = pd.DataFrame(columns=df_piv.columns)

    df_rank['index'] = df_piv['index']

    rank_ascending = ['mean', 'median', 'sum', 'std', 'var'] # 
    df_rank.loc[:, rank_ascending] = df_piv[rank_ascending].rank(method='dense', na_option='bottom', ascending=True)

    rank_descending = ['size']
    df_rank.loc[:, rank_descending] = df_piv[rank_descending].rank(method='dense', na_option='bottom', ascending=False)

    guess_rankings = dict()

    for desc_index in df_piv.columns.levels[1].to_list()[:-1]:
        df_rank.loc[:, ('rank_sum', desc_index)] = df_rank.loc[:, (rank_ascending + rank_descending, desc_index)].sum(axis=1)
        guesses = df_rank.loc[df_rank.loc[:, ('rank_sum', desc_index)].nsmallest(10).index, 'index'].to_list()

        for rank, guess in enumerate(guesses, start=1):
            if guess in guess_rankings.keys():
                guess_rankings[guess].append(rank)
            else:
                guess_rankings[guess] = [rank]

    top_guesses = [s[0] for s in sorted(guess_rankings.items(), key=lambda x: len(x[1]), reverse=True)][:10]
    alg_guesses.extend(top_guesses)
    #return [sample_actual, package_actual] + top_guesses
print(package_actual in alg_guesses)

True


In [131]:
df_rank

index  mean                         median        \
descriptor_actual                    1     2     3     4     5      1     2   
0                  0003_D02_S002  14.0  15.0  17.0  14.0  17.0   14.0  15.0   
1                  0005_D01_S001  14.0  15.0  17.0  14.0  12.0   14.0  15.0   
2                  0009_D01_S001  14.0   3.0  17.0  14.0  18.0   14.0   3.0   
3                  0019_D01_S001  14.0  15.0  17.0  10.0  18.0   14.0  15.0   
4                  0019_D01_S002  14.0  15.0  17.0  12.0  18.0   14.0  15.0   
..                           ...   ...   ...   ...   ...   ...    ...   ...   
63                 0823_D01_S001  14.0  15.0  16.0  14.0  18.0   14.0  15.0   
64                 0824_D01_S001  14.0  15.0  17.0  14.0   5.0   14.0  15.0   
65                 0834_D01_S001  14.0  11.0   2.0  14.0  18.0   14.0  11.0   
66                 0844_D01_S001   8.0  15.0  17.0  14.0  18.0    8.0  15.0   
67                 0860_D01_S001  14.0  15.0  17.0  14.0  11.0   14.0  15.0   

                               ...  var                     rank_sum        \
descriptor_actual     3     4  ...    1    2    3    4    5        1     2   
0                  17.0  14.0  ...  7.0  5.0  5.0  8.0  3.0     60.0  59.0   
1                  17.0  14.0  ...  7.0  5.0  5.0  8.0  3.0     60.0  59.0   
2                  17.0  14.0  ...  7.0  5.0  5.0  8.0  3.0     60.0  21.0   
3                  17.0  10.0  ...  7.0  5.0  5.0  3.0  3.0     60.0  59.0   
4                  17.0  12.0  ...  7.0  5.0  5.0  1.0  3.0     60.0  59.0   
..                  ...   ...  ...  ...  ...  ...  ...  ...      ...   ...   
63                 16.0  14.0  ...  7.0  5.0  5.0  8.0  3.0     60.0  59.0   
64                 17.0  14.0  ...  7.0  5.0  5.0  8.0  3.0     60.0  59.0   
65                  2.0  14.0  ...  7.0  5.0  5.0  8.0  3.0     60.0  42.0   
66                 17.0  14.0  ...  3.0  5.0  5.0  8.0  3.0     35.0  59.0   
67                 17.0  14.0  ...  7.0  5.0  5.0  8.0  3.0     60.0  59.0   

                                     
descriptor_actual     3     4     5  
0                  64.0  61.0  58.0  
1                  64.0  61.0  44.0  
2                  64.0  61.0  64.0  
3                  64.0  39.0  64.0  
4                  64.0  40.0  64.0  
..                  ...   ...   ...  
63                 57.0  61.0  64.0  
64                 64.0  61.0  23.0  
65                 18.0  61.0  64.0  
66                 64.0  61.0  64.0  
67                 64.0  61.0  41.0  

[68 rows x 41 columns]

In [ ]:
dasd = df.descriptor_actual.max() // 10

df['package_predicted'].value_counts().reset_index(name="count").query(f"count > {dasd}")

In [ ]:
sorted(df_piv['size'].isnull().sum(axis=1).unique())

In [ ]:
df_piv['size'].shape

In [ ]:
sample_actual

In [ ]:
df = dfs[2]

sample_actual = df.iloc[0].sample_actual
package_actual = df.iloc[0].package_actual
df[df.distance < 1].package_predicted.value_counts()

In [ ]:
df[df.algorithm == 'MI1'].descriptor_actual.max()